# Scientific Computing with Python (Second Edition)
# Chapter 09

*We start by importing all from Numpy. As explained in Chapter 01 the examples are written assuming this import is initially done.*


In [1]:
from numpy import *

## 9.1 The for statement

In [3]:
for s in ['a', 'b', 'c']:
    print(s)  # a b c

a
b
c


In [4]:
a=ones((3,5))
for k,el in ndenumerate(a):
    print(k,el)       
# prints something like this:  (1, 3) 1.0

(0, 0) 1.0
(0, 1) 1.0
(0, 2) 1.0
(0, 3) 1.0
(0, 4) 1.0
(1, 0) 1.0
(1, 1) 1.0
(1, 2) 1.0
(1, 3) 1.0
(1, 4) 1.0
(2, 0) 1.0
(2, 1) 1.0
(2, 2) 1.0
(2, 3) 1.0
(2, 4) 1.0


## 9.2 Controlling the flow inside the loop

*We prepare here the next code example from the book.*

In [6]:
def compute():
    return 1.e-5
tolerance = 1.e-3

In [7]:
maxIteration = 10000
for iteration in range(maxIteration):
    residual = compute() # some computation
    if residual < tolerance:
        break
else: # only executed if the for loop is not broken
    raise Exception("The algorithm did not converge")
print(f"The algorithm converged in {iteration + 1} steps")

The algorithm converged in 1 steps


## 9.3 Iterable objects

In [8]:
l=[1,2] 
li=l.__iter__() 

In [9]:
li.__next__() # returns 1 

1

In [10]:
li.__next__() # returns 2 

2

*Uncomment the next line to see the exception*

In [12]:
# li.__next__() # raises StopIteration exception 

### 9.3.1 Generators

In [13]:
def odd_numbers(n): 
    "generator for odd numbers less than n" 
    for k in range(n): 
        if k % 2 == 1: 
           yield k

In [14]:
g = odd_numbers(10)
for k in g:
    ...    # do something with k
    print(k)

1
3
5
7
9


In [15]:
for k in odd_numbers(10):
    ... # do something with k
    print(k)

1
3
5
7
9


### 9.3.2 Iterators are disposable

In [17]:
L = ['a', 'b', 'c']
iterator = iter(L)
list(iterator) # ['a', 'b', 'c']
list(iterator) # [] empty list, because the iterator is exhausted

[]

In [18]:
new_iterator = iter(L) # new iterator, ready to be used
list(new_iterator) # ['a', 'b', 'c']

['a', 'b', 'c']

In [19]:
g = odd_numbers(10)
for k in g:
    ... # do something with k
    print(k)

1
3
5
7
9


In [20]:
# now the iterator is exhausted:
for k in g: # nothing will happen!!
    ...
    print(k)

In [22]:
# to loop through it again, create a new one:
g = odd_numbers(10)
for k in g:
    ...
    print(k)

1
3
5
7
9


### 9.3.3 Iterator tools

In [23]:
A = ['a', 'b', 'c']
for iteration, x in enumerate(A):
    print(iteration, x)     # result: (0, 'a') (1, 'b') (2, 'c')

0 a
1 b
2 c


In [25]:
A = [0, 1, 2]
for elt in reversed(A):
    print(elt)      # result: 2 1 0

2
1
0


In [26]:
import itertools

In [27]:
for iteration in itertools.count():
    if iteration > 100:
       break # without this, the loop goes on forever
       print(f'integer: {iteration}')
       # prints the 100 first integer

In [28]:
from itertools import count, islice
for iteration in islice(count(), 10): 
     # same effect as range(10)
        print (iteration)

0
1
2
3
4
5
6
7
8
9


In [29]:
list(islice(count(), 10))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [31]:
def odd_numbers():
    k=-1
    while True:  # this makes it an infinite generator
        k+=1
        if k%2==1:
           yield k

In [32]:
list(itertools.islice(odd_numbers(),10,30,8)) 
# returns [21, 37, 53]

[21, 37, 53]

### 9.3.4 Generators of recursive sequences


In [33]:
def fibonacci(u0, u1):
    """
    Infinite generator of the Fibonacci sequence.
    """
    yield u0
    yield u1
    while True:
        u0, u1 = u1, u1 + u0  
        # we shifted the elements and compute the new one
        yield u1

In [35]:
# sequence of the 10 first Fibonacci numbers:
list(itertools.islice(fibonacci(0, 1), 10))

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34]

In [36]:
def arithmetic_geometric_mean(a, b):
    """
    Generator for the arithmetic and geometric mean
    a, b initial values
    """ 
    while True:    # infinite loop
         a, b = (a+b)/2, sqrt(a*b)
         yield a, b

In [37]:
def elliptic_integral(k, tolerance=1.e-5):
    """
    Compute an elliptic integral of the first kind.
    """
    a_0, b_0 = 1., sqrt(1-k**2)
    for a, b in arithmetic_geometric_mean(a_0, b_0):
        if abs(a-b) < tolerance:
            return pi/(2*a)

In [38]:
from itertools import islice
def elliptic_integral(k, tolerance=1e-5, maxiter=100):
    """
    Compute an elliptic integral of the first kind.
    """
    a_0, b_0 = 1., sqrt(1-k**2)
    for a, b in islice(arithmetic_geometric_mean(a_0, b_0), 
                                                  maxiter):
        if abs(a-b) < tolerance:
            return pi/(2*a)
    else:
        raise Exception("Algorithm did not converge")

In [39]:
def pendulum_period(L, theta, g=9.81):
    return 4*sqrt(L/g)*elliptic_integral(sin(theta/2))

In [40]:
pendulum_period(1,pi/2)

2.367841947461051

In [41]:
def Euler_accelerate(sequence):
    """
    Accelerate the iterator in the variable `sequence`.
    """
    s0 = sequence.__next__() # Si
    s1 = sequence.__next__() # Si+1
    s2 = sequence.__next__() # Si+2
    while True:
        yield s0 - ((s1 - s0)**2)/(s2 - 2*s1 + s0)
        s0, s1, s2 = s1, s2, sequence.__next__()

In [42]:
def pi_series():
    sum = 0.
    j = 1
    for i in itertools.cycle([1, -1]):
        yield sum
        sum += i/j
        j += 2

In [43]:
N=15
list(itertools.islice(Euler_accelerate(pi_series()), N))

[0.75,
 0.7916666666666667,
 0.7833333333333334,
 0.7863095238095239,
 0.784920634920635,
 0.7856782106782109,
 0.7852203352203354,
 0.7855179542679545,
 0.7853137059019414,
 0.7854599047323508,
 0.7853516796241258,
 0.7854340248151667,
 0.7853699222510641,
 0.7854207973019391,
 0.7853797463988194]

## 9.4 List-filling patterns
### 9.4.1 List filling with the append method


In [47]:
L = []
n=3
for k in range(n):
    # call various functions here
    # that compute "result"
    result = k
    L.append(result)
L

[0, 1, 2]

### 9.4.2 List from iterators

In [48]:
def result_iterator():
    for k in itertools.count(): # infinite iterator
        # call various functions here
        # that t lists compute "result"
        result = k
        yield result

In [51]:
L = list(itertools.islice(result_iterator(), n)) # no append needed!
L

[0, 1, 2]

The next statement requires the built-in function `sum`. As we previously imported numpy by 
`from numpy import *` the next statement uses numpy.sum and will not give us the result described in the book.
Deleting the imported `sum` gives us the built-in `sum` back

In [91]:
del sum

In [92]:
sum(itertools.islice(result_iterator(), n))

190

In [93]:
L=list(itertools.takewhile(lambda x: abs(x) > 1.e-8, result_iterator()))

In [94]:
L

[]

### 9.4.3 Storing generated values

In [95]:
import itertools
def power_sequence(u0):
    u = u0
    while True:
        yield u
        u = u**2

*To see the OverflowError exception, uncomment the next line.*

In [96]:
# list(itertools.islice(power_sequence(2.), 20))

In [97]:
generator = power_sequence(2.)
L = []
for iteration in range(20):
    try:
        L.append(next(generator))
    except Exception:
        break
L

[2.0,
 4.0,
 16.0,
 256.0,
 65536.0,
 4294967296.0,
 1.8446744073709552e+19,
 3.402823669209385e+38,
 1.157920892373162e+77,
 1.3407807929942597e+154]

## 9.5 When iterators behave as lists
### 9.5.1 Generator expressions


In [98]:
g = (n for n in range(1000) if not n % 100)
# generator for  100, 200, ... , 900

*See the comment above, concerning built-in functions and numpy's `sum`*

In [99]:
sum(n for n in range(1000) if not n % 100) # returns 4500 

4500

In [100]:
N=20
s=2
sum(1/n**s for n in itertools.islice(itertools.count(1), N))


1.5961632439130233

In [101]:
list(itertools.islice(itertools.count(1), N))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

In [102]:
def generate_zeta(s):
    for n in itertools.count(1):
        yield 1/n**s

In [103]:
def zeta(N, s):
    # make sure that you do not use the scipy.sum here
    return sum(itertools.islice(generate_zeta(s), N))

In [104]:
zeta(20,2)

1.5961632439130233

### 9.5.2 Zipping iterators

*We first create for the example two iterators:*

In [108]:
def odd_numbers():
    k=-1
    while True:  # this makes it an infinite generator
        k+=1
        if k%2==1:
           yield k
def even_numbers():
    k=0
    while True:  # this makes it an infinite generator
        k+=1
        if k%2==0:
           yield k        

In [109]:
xg = itertools.islice(odd_numbers(),10)  # some iterator
yg = itertools.islice(even_numbers(),10)  # another iterator

for x, y in zip(xg, yg):
    print(x, y)

1 2
3 4
5 6
7 8
9 10
11 12
13 14
15 16
17 18
19 20


## 9.6 Iterator objects


In [112]:
class OdeStore:
    """
    Class to store results of ode computations
    """
    def __init__(self, data):
        "data is a list of the form [[t0, u0], [t1, u1],...]"
        self.data = data
    
    def __iter__(self):
        "By default, we iterate on the values u0, u1,..."
        for t, u in self.data:
            yield u

store = OdeStore([[0, 1], [0.1, 1.1], [0.2, 1.3]])
for u in store:
    print(u)
# result: 1, 1.1, 1.3
list(store) # [1, 1.1, 1.3]

1
1.1
1.3


[1, 1.1, 1.3]

## 9.7 Infinite iterations
### 9.7.1 The while loop
*no complete code in this subsecdtion*
### 9.7.2 Recursion



In [115]:
def f(N):
    if N == 0: 
        return 0
    return f(N-1)